In [1]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

device = "cpu"  # Set to CPU explicitly
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

text = "Hello, Hugging Face!"
tokens = tokenizer(text, return_tensors="pt").to(device)

generated_tokens = model.generate(**tokens, max_length=50)
generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

print(generated_text)


c:\Users\suvom\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, Hugging Face!

I'm not sure if you've heard of the "Hugging Face" meme, but it's a meme that's been around for a while now. It's a meme that's been around for a


In [2]:
prompt = "Once upon a time in a distant land,"
tokens = tokenizer(prompt, return_tensors="pt")

generated_tokens = model.generate(**tokens, max_length=50)
generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time in a distant land, the sun was shining, and the moon was shining. The sun was shining, and the moon was shining. The sun was shining, and the moon was shining. The sun was shining, and the moon


In [3]:
generated_tokens = model.generate(**tokens, 
                                  max_length=50, 
                                  do_sample=True, 
                                  top_k=50,  # Limits to top 50 probable words
                                  top_p=0.9, # Adjusts probability threshold
                                  num_beams=5) # Beam search for better sequences

generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time in a distant land, there was a man who had been sent by God to save the world. He was a man who had been sent by God to save the world. He was a man who had been sent by God to


In [4]:
from datasets import load_dataset

dataset = load_dataset('imdb')

def preprocess(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenizer.pad_token = tokenizer.eos_token 
tokens = tokenizer(text, return_tensors="pt", padding=True)
tokenized_dataset = dataset.map(preprocess, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

model_name = "distilgpt2"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenizer.pad_token = tokenizer.eos_token 
tokens = tokenizer(text, return_tensors="pt", padding=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train model
trainer.train()
